# Kaggle RSNA Intracranial Hemorrhage Detection

Ce notebook utilise les données de la compétition [Kaggle RSNA Intracranial Hemorrhage Detection](https://www.kaggle.com/competitions/rsna-intracranial-hemorrhage-detection/overview) pour entraîner un modèle de diffusion stable à générer des images médicales.

## Sommaire

- [0. Importation des bibliothèques](#0-Importation-des-bibliothèques)
- [1. Chargement et prétraitement des données](#1-Chargement-et-prétraitement-des-données)
- [2. Définition du modèle de diffusion](#2-Définition-du-modèle-de-diffusion)
- [3. Entraînement du modèle](#3-Entraînement-du-modèle)
- [4. Génération d'images](#4-Génération-d'images)
- [5. Conclusion](#5-Conclusion)

## 0. Importation des bibliothèques

In [35]:
import pandas as pd
from typing import List
import subprocess
from pathlib import Path

## 1. Chargement et prétraitement des données

Nous allons d'abord charger la liste des identifiants et labels du jeu de données d'entraînement. L'API Kaggle est une option, ou sinon vous pouvez télécharger le fichier CSV manuellement depuis la [page de la compétition](https://www.kaggle.com/competitions/rsna-intracranial-hemorrhage-detection/data) et le placer dans le répertoire `../data/rsna-intracranial-hemorrhage-detection`.  

In [21]:
COMPETITION_NAME = "rsna-intracranial-hemorrhage-detection"
DATA_DIRECTORY = "../data/" + COMPETITION_NAME

In [22]:
train_dataset_listing = pd.read_csv(f"{DATA_DIRECTORY}/stage_2_train.csv")

In [27]:
train_dataset_listing.head()

,ID,Label
0,ID_12cadc6af_epidural,0
1,ID_12cadc6af_intraparenchymal,0
2,ID_12cadc6af_intraventricular,0
3,ID_12cadc6af_subarachnoid,0
4,ID_12cadc6af_subdural,0


In [28]:
train_dataset_listing['Label'].unique()

array([0, 1])

On voit donc que les "ID" sont de la forme "ID_ImageID_HemorrhageType", et que la colonne "Label" ne peut prendre que les valeurs 0 ou 1. Nous allons donc créer une nouvelle colonne "ImageID" qui ne contiendra que la partie "ImageID" de l'ID, et une colonne "HemorrhageType" qui contiendra la partie "HemorrhageType".

In [29]:
train_dataset_listing['HemorrhageType'] = train_dataset_listing['ID'].str.split("_", n = 3, expand = True)[2]
train_dataset_listing['ImageID'] = train_dataset_listing['ID'].str.split("_", n = 3, expand = True)[1]


In [30]:
train_dataset_listing.head()

,ID,Label,HemorrhageType,ImageID
0,ID_12cadc6af_epidural,0,epidural,12cadc6af
1,ID_12cadc6af_intraparenchymal,0,intraparenchymal,12cadc6af
2,ID_12cadc6af_intraventricular,0,intraventricular,12cadc6af
3,ID_12cadc6af_subarachnoid,0,subarachnoid,12cadc6af
4,ID_12cadc6af_subdural,0,subdural,12cadc6af


Partant de là, on peut maintenant faire la liste des ImageID uniques :

In [31]:
unique_image_ids = train_dataset_listing['ImageID'].unique()
print(f"Number of unique ImageIDs: {len(unique_image_ids)}")

Number of unique ImageIDs: 752803


Nous allons pouvoir maintenant charger les images DICOM correspondantes à ces ImageID. Pour cela, nous allons créer une fonction qui, pour une liste d'ImageIDs donnée, charge l'image DICOM correspondante, dans un dossier spécifié.

In [37]:
def download_images(image_ids: List[str],
                    destination_directory: str = DATA_DIRECTORY):
    """
    Download DICOM files from Kaggle CLI.

    Parameters
    ----------
    image_ids : List[str]
        List of DICOM IDs to download (without extension)
    destination_directory : str
        Path to the directory where to save the downloaded files
    """
    Path(destination_directory).mkdir(parents=True, exist_ok=True)

    for iid in image_ids:
        remote_path = f"rsna-intracranial-hemorrhage-detection/stage_2_train/ID_{iid}.dcm"
        subprocess.run(
            [
                "kaggle", "competitions", "download",
                "-c", COMPETITION_NAME,
                "-f", remote_path,
                "-p", destination_directory
            ],
            check=True
        )

Testons cette fonction en téléchargeant les images correspondant aux 5 premiers ImageIDs uniques :

In [38]:
first_5_image_ids = unique_image_ids[:5]
download_images(first_5_image_ids)

100%|██████████| 513k/513k [00:00<00:00, 430MB/s]


100%|██████████| 513k/513k [00:00<00:00, 792MB/s]


100%|██████████| 513k/513k [00:00<00:00, 560MB/s]


100%|██████████| 513k/513k [00:00<00:00, 1.33GB/s]


100%|██████████| 513k/513k [00:00<00:00, 748MB/s]
